激活函数：引入非线性+确保梯度传递(消失/爆炸)+能否收敛

In [ ]:
# 特征工程

### VGG
相比AlexNet框架更规则，更多的模块化
得出更深更窄效果更好
特别占内存
VGG-N：N包括了全连接层
### NiN
网络中的网络：完全不要full
NiN块：使用1*1的conv代替full
架构：输入通道数是类别数，交替使用NiN块+Pool(stride=2),最后用全局avgPool代替full得到输出
    不易overfit，更少参数个数
### GoogLeNet
9个inception块
Inception块：四个路径从不同层面抽取信息，在输出通道维合并 
更少参数个数 + 计算复杂度

### 这些都是resnet的灵感？
乘法变加法

In [ ]:
#GoogLeNet 实现

批量归一化
    全连接层作用在特征维，卷积层作用在通道维；
    全&卷输出上，全&卷输入上，激活函数前
    本质：向每个小批量内加入噪音，不用和dropout混用
    会加速收敛(可用更大的lr)，但不改变精度

如果一个数据集有 n 类，批量大小不要超过 10*n

In [ ]:
#调包
nn.BatchNormnd('输入维度')  # 选几d & 输入维度

In [ ]:
#判断张量是否在同设备上，设备对齐
x = torch.randn(2, 2)
X = torch.randn(2, 2).to("cuda")

print(x.device)  # cpu
print(X.device)  # cuda:0

print(x.device != X.device)  # True

torch.backends.cudnn.benchmark = True   #（自动选择最快卷积算法）

#小批量多GPU最后把梯度累加即可

AMP自动混合精度
GradScaler 防止FP16时梯度过小导致下溢(变0)

In [ ]:
# AMP + GradScaler
from torch.cuda.amp import autocast, GradScaler

model = MyModel().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scaler = GradScaler()

for data, targets in dataloader:
    data, targets = data.cuda(), targets.cuda()
    optimizer.zero_grad()

    with autocast():  # 自动使用混合精度
        outputs = model(data)
        loss = loss_fn(outputs, targets)

    scaler.scale(loss).backward()      # 放大 loss 并反向传播
    scaler.step(optimizer)             # 等价于 optimizer.step()
    scaler.update()                    # 动态调整缩放因子



In [ ]:
# 数据增广（P36）